In [21]:
import pandas as pd
import re
import ast

In [86]:
df = pd.read_csv('train.csv')

In [87]:
# Function to extract unique values followed after "name" field
def extract_unique_names(text):
    pattern = r'"name":\s*"(.*?)"'
    return re.findall(pattern, text)

# Apply the function to each row in the "system" column
df['function_calls'] = df['system'].apply(extract_unique_names)

In [88]:
# Function to extract and convert dictionaries from the string
def extract_dicts(row):
    # Find the first occurrence of '{'
    index = row['system'].find('{')
    if index != -1:
        # Extract everything from the first '{' onwards
        remaining_str = row['system'][index:]
        # Split the string by newline to separate individual dictionaries
        dict_strings = remaining_str.split('\n\n')
        dicts = []
        for dict_str in dict_strings:
            dict_str = dict_str.strip()
            if dict_str:  # Ensure the string is not empty
                try:
                    # Convert string to dictionary
                    dict_obj = ast.literal_eval(dict_str)
                    dicts.append(dict_obj)
                except (ValueError, SyntaxError) as e:
                    continue
        return pd.Series([dicts], index=['extracted_dicts'])
    else:
        # Handle the case where no dictionary is present
        return pd.Series([[]], index=['extracted_dicts'])
# Apply the function to each row
df['extracted_dicts'] = df.apply(extract_dicts, axis=1)['extracted_dicts']

In [90]:
# Flatten the lists of function calls and corresponding dictionaries
function_call_dicts = []
for index, row in df.iterrows():
    for func, dict_obj in zip(row['function_calls'], row['extracted_dicts']):
        function_call_dicts.append((func, dict_obj))

# Create a new DataFrame with unique function calls and their corresponding dictionaries
unique_functions = pd.DataFrame(function_call_dicts, columns=['function_call', 'dictionary'])
unique_functions = unique_functions.drop_duplicates(subset='function_call')
unique_functions['dictionary'] = unique_functions['dictionary'].apply(lambda x: str(x))

# Set function_call as the index
unique_functions.set_index('function_call', inplace=True)

# Display the new DataFrame
unique_functions

,dictionary
function_call,
get_exchange_rate,"{'name': 'get_exchange_rate', 'description': '..."
get_news_headlines,"{'name': 'get_news_headlines', 'description': ..."
generate_password,"{'name': 'generate_password', 'description': '..."
create_task,"{'name': 'create_task', 'description': 'Create..."
calculate_median,"{'name': 'calculate_median', 'description': 'C..."
...,...
get_wikipedia_summary,"{'name': 'get_wikipedia_summary', 'description..."
verify_user,"{'name': 'verify_user', 'description': 'Verify..."
find_local_restaurants,"{'name': 'find_local_restaurants', 'descriptio..."


In [91]:
unique_functions.to_csv('fc.csv')